In [1]:
from selenium import webdriver
import time
import requests
import pandas as pd
import openpyxl
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from tqdm import tqdm_notebook


from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [6]:
# path_는 사용자에 따라 변경

path_ = './chromedriver/chromedriver'

url = 'https://www.dovechannel.com/browse-movie/148/free/'

start = time.time()

driver = webdriver.Chrome(path_)

driver.get(url)

time.sleep(3)

driver.maximize_window()

time.sleep(10)

soup = BeautifulSoup(driver.page_source)

# 값 list들 생성

dovechannel_id_list = []
https_list = []
title_list = []
synopsis_list = []
director_list = []
genres_list = []
starring_list = []
release_list = []

# 사이트에 lock이 걸려있는 것과 걸려있지 않은 것이 있고, lock이 있다면 유료인 사이트이므로 들어가지 않아야 함

hrefs = soup.find_all('div', href=True)

for i in hrefs:
    if not i.find(class_ = 'lock'):
        href = i.attrs['href']
        https_list.append('http://dovechannel.com' + href.strip())

# 긁어온 값들 중 중복된 값이 다수 존재해 중복을 제거        
        
https_list_no_duplicate = list(set(https_list))

# dovechannel에서 크롤링하기

for i in https_list_no_duplicate:
    # 거슬리면 제거
    print(i + ' working...')
    
    driver.get(i)
    
    # 사이트의 로딩이 느려서 20초 부여했지만 인터넷 상황이 괜찮다면 더 짧아도 괜찮음
    time.sleep(20)

    soup = BeautifulSoup(driver.page_source)
    
    title_list.append(soup.find('h2', class_ = "title-text text-white").get_text().strip())
    synopsis_list.append(soup.find('p', class_ = 'para-text block-with-text').get_text().strip())
    
    s_d = soup.find_all('div', class_ = 'w-100 d-block')
    
    # 감독, 장르, 배우들이 따로 나뉘어져 있지 않고 하나의 태그 안에 존재해서, 각 단어들이 처음 나타나는 지점을 기록하고 slicing 이용하여 데이터 뽑기
    director_loc = s_d[3].get_text().find('Director')
    genres_loc = s_d[3].get_text().find('Genres')
    starring_loc = s_d[3].get_text().find('Starring')

    director_list.append(s_d[3].get_text()[director_loc + 10:genres_loc].strip())
    genres_list.append(s_d[3].get_text()[genres_loc + 8:starring_loc].strip())
    starring_list.append(s_d[3].get_text()[starring_loc + 10:].strip())

print('Crawling Done!')

# https 안에 uid가 존재해서 마찬가지로 slicing 이용하여 uid 추출

start_loc = []

for i in https_list_no_duplicate:
    for j in re.finditer('/',i):
        start_loc.append(j.start())
    dovechannel_id_list.append(i[start_loc[3]+1:start_loc[4]])
    start_loc = []

# release 년도 추출    
    
tmp_years = []
driver = webdriver.Chrome(path_)

for i in tqdm_notebook(range(len(title_list))):
    driver.get('https://www.google.com/search?q='+ ' ' + title_list[i] + ' ' + director_list[i])
    print('https://www.google.com/search?q='+ ' ' + title_list[i] + ' ' + director_list[i])
    time.sleep(5)
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')
        results = soup.find('span', class_="LrzXr kno-fv").text
        print(results)
        tmp_years.append(re.findall("\d\d\d\d",results))
        print("final: ", re.findall("\d\d\d\d",results))
    except Exception as err:
        tmp_years.append("None")
        print("final: None")
    release_list.append(str(tmp_years[i]).replace("[","").replace("]","").replace("'",""))

# 데이터프레임 형식으로 추출

items = pd.DataFrame({'dovechannel_id' : dovechannel_id_list, 'title' : title_list, 'release' : release_list, 
                      'genres' : genres_list, 'director' : director_list, 'actor' : starring_list,
                      'country' : 'None', 'runtime' : 'None', 'production' : 'None', 'overview' : synopsis_list,
                      'url' : https_list_no_duplicate})
    
# 비어있거나 '-'인 값들을 None으로 변경   
    
items.replace({'release' : ''}, {'release' : 'None'}, inplace = True)
items.replace({'director' : '-'}, {'director' : 'None'}, inplace = True)
items.replace({'actor' : '-'}, {'actor' : 'None'}, inplace = True)
        
driver.close()
    
print('Spend time : ', time.time() - start)

http://dovechannel.com/details-movie/CEG10019213/the-book-of-genesis working...
http://dovechannel.com/details-movie/CEG10015223/walk-by-faith working...
http://dovechannel.com/details-movie/CEG10015959/the-dogfather working...
http://dovechannel.com/details-movie/CEG10014920/john-wayne-in-stagecoach-run-in-color working...
http://dovechannel.com/details-movie/CEG10022951/george! working...
http://dovechannel.com/details-movie/CEG10016020/full-of-grace working...
http://dovechannel.com/details-movie/CEG10019914/this-is-the-army working...
http://dovechannel.com/details-movie/CEG10019904/his-girl-friday working...
http://dovechannel.com/details-movie/CEG10020711/just-let-go working...
http://dovechannel.com/details-movie/CEG10019143/different-drummers working...
http://dovechannel.com/details-movie/NNVG10008106/the-best-of-laurel-and-hardy-(in-color-&-restored) working...
http://dovechannel.com/details-movie/CEG10020489/bobby-jones-stroke-of-genius:-special-inspirational-edition working

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


https://www.google.com/search?q= The Book of Genesis Craig S. Cunningham
크레이그 S. 커닝햄
final:  []
https://www.google.com/search?q= Walk By Faith Trina Brown, LaMarcus Tinker
final: None
https://www.google.com/search?q= The Dogfather Richard Boddington
2010년 3월 12일
final:  ['2010']
https://www.google.com/search?q= John Wayne in Stagecoach Run IN COLOR Mack V. Wright
1936년 7월 6일 (미국)
final:  ['1936']
https://www.google.com/search?q= George! Wallace C. Bennett
월레스 C. 베넷
final:  []
https://www.google.com/search?q= Full of Grace Andrew Hyatt
2015년 10월
final:  ['2015']
https://www.google.com/search?q= This is the Army Michael Curtiz
1943년 7월 29일
final:  ['1943']
https://www.google.com/search?q= His Girl Friday Howard Hawks
1940년 1월 18일 (미국)
final:  ['1940']
https://www.google.com/search?q= Just Let Go Christopher S. Clark, Patrick Henry Parker
2015년 10월 9일
final:  ['2015']
https://www.google.com/search?q= Different Drummers Don Caron
2009년
final:  ['2009']
https://www.google.com/search?q= The 

In [7]:
items.to_excel('Dovechannel_Crawling.xlsx')